In [ ]:
%load_ext autoreload
%autoreload 2

# Create pool using custom image

In this case we use the custom image from SCHISM. the image was created in eastus so a pre-requisite is to make sure the image is replicated to the same region as the batch account (dwrmodelingbatch) in this case which is in westus2

Note: Unsuccessful in getting the image published from eastus visible in westus2 so moving the batch account to eastus. This also saves in terms of replication of the images which can take a while and incur costs.

Note: Also unsuccessfull in creating via Batch Explorer. Recommend creating via Web Portal

```
Operating system
Linux (centos 7.9.2009)

Publisher
DWR_MSO

Offer
SCHISM

Plan
5_8

VM image definition
SCHISM

VM generation
V2

VM architecture
x64
```

A batch account was created in eastus under the dwrbdoschism_rg group to be right next to the image and storage account regions and availability zones.

## Imports

In [ ]:
from dmsbatch import create_batch_client, create_blob_client
import datetime

## First create a batch client from the config file

In [ ]:
client = create_batch_client('../tests/data/schism_combine.config')
blob_client = create_blob_client('../tests/data/schism_combine.config')

## Find the image reference

The code below gets the supported images. However I was unsuccessfull in using the image reference to create a pool 

In [ ]:
import azure.batch.models as batchmodels

options = batchmodels.AccountListSupportedImagesOptions()

#images = client.batch_client.account.list_supported_images(account_list_supported_images_options=options)
images = client.batch_client.account.list_supported_images()

skus_to_use = [(image.node_agent_sku_id, image.image_reference) for image in images]

#for sku_id, _ in skus_to_use: print(sku_id)

len(skus_to_use)

#client.select_latest_verified_vm_image_with_node_agent_sku('DWR_MSO','SCHISM','5_8')

# bay-delta-schism2-v58 (eli/simulations/hindcast_clinic2/outputs/schout_0000_1/**/*)	

image_resource_id = '/subscriptions/c15db114-26b5-454c-b8f4-8a5eb5f16796/resourceGroups/dwrbdo_dcp_rg/providers/Microsoft.Compute/galleries/dwrmso/images/SCHISM/versions/5.8.0'
image_reference = batchmodels.ImageReference(publisher='DWR_MSO', offer='SCHISM', sku='5_8', version='5.8.0', virtual_machine_image_id=image_resource_id)
vmconfig = batchmodels.VirtualMachineConfiguration(image_reference = image_reference, node_agent_sku_id='batch.node.centos')

## Application packages
To copy large files and programs it is best to zip (or targz) them and upload them as application packages

Application packages are setup separately in either azure management apis or from the web console or cli tool

These are referenced here by their name and version
e.g. DSM2, python and other programs

One extra field (last one) is the path within the zip file where the executables can be found. These are used later to setup the PATH varible

### Show vms available

https://docs.microsoft.com/en-us/azure/virtual-machines/fsv2-series

In [ ]:
#display(client.skus_available(filter=None))

### Create or resize existing pool
If the pool doesn't exist it will create it
If the pool exists, it will resize to the second arg

#### Specify VMs and number of cores and hosts
You can use az to figure out what vms are available
```
az batch location list-skus --location westus --output tsv
```


#### Use a pool start task to install libraries

In [ ]:
pool_name='schism58' # from custom image with schism v5.8
num_hosts=1
# Set to False as I have only been able to find the image via BatchExplorer and not programmatically.
if False:
    pool_start_cmds = ['printenv']
    client.wrap_commands_in_shell(pool_start_cmds, ostype='linux')


    vm_core_map={'standard_hc44rs':44,'standard_hb120rs_v2':120, 'Standard_F2s_v2': 2 , 'Standard_D8as_v4': 8}

    vm_size= 'Standard_D8as_v4' #'standard_hc44rs' #'standard_hb120rs_v2',

    #### Create pool with the OS image and enable internode communication

    image_resource_id = '/subscriptions/c15db114-26b5-454c-b8f4-8a5eb5f16796/resourceGroups/dwrbdo_dcp_rg/providers/Microsoft.Compute/galleries/dwrmso/images/SCHISM/versions/5.8.0'
    #publisher='DWR_MSO', offer='SCHISM', sku='5_8', version='5.8.0', # this doesn't work so using resource id directly
    image_reference = batchmodels.ImageReference(virtual_machine_image_id=image_resource_id)
    pool_created = client.create_pool(pool_name,
                                    num_hosts,
                                    vm_size=vm_size,
                                    tasks_per_vm= vm_core_map[vm_size],
                                    os_image_reference=('batch.node.centos 7',image_reference), # ? can we use our custom image
                                    start_task_cmd=client.wrap_commands_in_shell(pool_start_cmds, ostype='linux'),
                                    start_task_admin=True,
                                    elevation_level='admin',
                                    enable_inter_node_communication=False,
                                    wait_for_success=True
                      )
else:
    pool_created = False

In [ ]:
#if not pool_created: client.resize_pool(pool_name,num_hosts)

In [ ]:
#client.wait_for_pool_nodes(pool_name)

### Get container url for where data exists with read/write permissions


In [ ]:
container_name='bay-delta-schism2-v58'

import dmsbatch
permissions = dmsbatch.commands.ContainerSasPermissions(read=True, write=True, list=True)
output_dir_sas_url = blob_client.get_container_sas_url(container_name, permissions)
container_url, sas_token = tuple(str.split(output_dir_sas_url,'?'))

container_url, sas_token

### Create a startup task for job

This is used to copy an information needed by all tasks and place them in a shared location

In [ ]:
# for az batch monitoring. See https://github.com/Azure/batch-insights#centos
# wget  -O - https://raw.githubusercontent.com/Azure/batch-insights/master/centos.sh | bash

local_to_global_url = container_url+f'/eli/simulations/hindcast_clinic2/outputs/*' 

cmd_string = client.wrap_commands_in_shell([
    f'azcopy cp "{local_to_global_url}?{sas_token}" "." --include-pattern "local_to_global*"', # --recursive doesn't work well with patterns
    'cp -r local_to_global* ${AZ_BATCH_NODE_SHARED_DIR}',
   ], ostype='linux')
copy_local_global_task = client.create_task('copy_local_global_task',
                                 cmd_string.replace('/bin/bash','/usr/bin/bash'))

copy_local_global_task = client.create_task('local_to_global_copy_task',
                                             cmd_string.replace('/bin/bash','/usr/bin/bash'))

### Create job on pool or fail if it exists
Jobs are containers of tasks (things that run on nodes (machines) in the pool). If this exists, the next line will fail

In [ ]:
job_name='schismjobs'
client.create_job(job_name,pool_name,prep_task=copy_local_global_task)

### Create a task
This uses the application package as pre -set up. If not, create one https://docs.microsoft.com/en-us/azure/batch/batch-application-packages

Create task name (unique) and the command string

In [ ]:
def create_task_for_day(day_index, ncombines=0):
    '''
    day_index is the day to combine
    ncombines (if 0 it will have to scan the pattern so will be slower, otherwise give the number of processors to combine)
    '''
    # Input files
    nc_input_url = f'{container_url}/eli/simulations/hindcast_clinic2/outputs/*'
    nc_blob_path = 'schout*_${day_index}.nc' # the day_index is available in the bash shell
    #Specify location to download script to run from
    combine_script_spec = client.create_input_file_spec('batch','schism_combine_one.sh','.')
    #Specify output files as spec references - Didn't work when clean up is needed after upload.
    #output_file = client.create_output_file_spec(f'schout_{day_index}.nc',
    #                                            output_dir_sas_url, 
    #                                            blob_path=f'eli/simulations/hindcast_clinic2/outputs.combined/schout_{day_index}.nc')
    #
    task_name = f'combine_output11_{day_index}'
    cmd_string = client.wrap_commands_in_shell([
        'cp -r ${AZ_BATCH_NODE_SHARED_DIR}/local_to_global_* .',
        f'/bin/bash -f schism_combine_one.sh "{nc_input_url}" "{nc_blob_path}" "{sas_token}" {day_index} {ncombines}',
        f'azcopy cp "schout_{day_index}.nc" "{container_url}/eli/simulations/hindcast_clinic2/outputs.combined/schout_{day_index}.nc?{sas_token}"',
        'rm -rf *.nc local_to_global*'
       ], ostype='linux')
    #Create task with all the above information
    schism_task = client.create_task(task_name,
                                     cmd_string,#.replace('/bin/bash','/usr/bin/bash'),
                                     resource_files=[combine_script_spec])#output_files=[output_file])
    return schism_task

### Next submit the task and wait 

In [ ]:
# max is 3948 for hindcast2, and 172 processors to combine
#client.submit_tasks(job_name,[create_task_for_day(day_index, 172) for day_index in range(1,3949)])
client.submit_tasks(job_name,[create_task_for_day(day_index, 172) for day_index in range(1,2)])

In [ ]:
#client.wait_for_tasks_to_complete(job_name,timeout=datetime.timedelta(seconds=120))

## Finally resize the pool to 0 to save costs

In [ ]:
#client.resize_pool(pool_name,0)